# Using a pretrained network as a feature extractor

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [29]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

In [24]:
# Download the pretrained model with it's weights but freeze it's classifier layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [25]:
# use the features it has already extracted without finding new features
for layers in base_model.layers:
    layers.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [42]:
# add our own classifier layer
base_model_last_layer = base_model.get_layer('block5_pool') # get last layer
base_model_as_output = base_model_last_layer.output # getting it's output value

flatten_base_model = Flatten()(base_model_as_output)
output_layer = Dense(10, activation='softmax', name='softmax')(flatten_base_model)

# the new model with the extracted features
new_model = Model(inputs=base_model.input, outputs=output_layer)
new_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0  

In [43]:
# this is just a test

# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.utils import to_categorical

# (X_train, y_train), (X_test, y_test) = cifar10.load_data()

# X_train = ((X_train - np.mean(X_train))/np.std(X_train)).astype('float32')
# X_test = ((X_test - np.mean(X_train))/np.std(X_train)).astype('float32')

# y_train = to_categorical(y_train, len(np.unique(y_train)))
# y_test = to_categorical(y_test, len(np.unique(y_test)))



# new_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
# history = new_model.fit(X_train[:40000], y_train[:40000],
#                         validation_data=(X_train[40000:], y_train[40000:]),
#                         epochs=1, batch_size=128)

313/313 [==============================] - 641s 2s/step - loss: 1.7596 - accuracy: 0.3875 - val_loss: 1.2323 - val_accuracy: 0.5855
